### Strassen’s method 스트라센 메소드(알고리즘)

정사각 행렬 곱 $C=A\cdot B$를 **더 적은 곱셈 횟수**로 계산해서 이론적으로 더 빠르게 만드는 분할정복 알고리즘

일반적인 3중 루프는 $O(n^3)$ 지만 스트라센 알고리즘을 사용하면 $T(n)=O(n^{log_{2}​7})≈O(n^{2.807})$ 로 낮아짐

#### 아이디어
일반적인 $2\times2$ 블록 행렬 곱은 블록 곱셈이 **8번** 필요  
행렬을 4개 블록으로 나눔(각 블록은 $n/2 \times n/2$)

$A=\begin{pmatrix} \ a_{11}&a_{12}\\ a_{21}&a_{22}\end{pmatrix},\  B=\begin{pmatrix} \ b_{11}&b_{12}\\ b_{21}&b_{22}\end{pmatrix}$

일반 곱은  
$\begin{aligned}c_{11}=a_{11}\cdot b_{11}+a_{12}\cdot b_{21}\\    
c_{12}=a_{11}\cdot b_{12}+a_{12}\cdot b_{22}\\
c_{21}=a_{21}\cdot b_{11}+a_{22}\cdot b_{21}\\
c_{22}=a_{21}\cdot b_{12}+a_{22}\cdot b_{22}\end{aligned}$  

총 8번의 곱하기를 게산해야 하지만, Strassen은 곱셈을 7번만 실행


$\begin{aligned} M_1 &= (A_{11}+A_{22})(B_{11}+B_{22})\\ M_2 &= (A_{21}+A_{22})B_{11}\\ M_3 &= A_{11}(B_{12}-B_{22})\\ M_4 &= A_{22}(B_{21}-B_{11})\\ M_5 &= (A_{11}+A_{12})B_{22}\\ M_6 &= (A_{21}-A_{11})(B_{11}+B_{12})\\ M_7 &= (A_{12}-A_{22})(B_{21}+B_{22}) \end{aligned}$

그다음 결과 블록을 덧셈/뺄셈으로 조립:

$\begin{aligned} C_{11} &= M_1 + M_4 - M_5 + M_7\\ C_{12} &= M_3 + M_5\\ C_{21} &= M_2 + M_4\\ C_{22} &= M_1 - M_2 + M_3 + M_6 \end{aligned}$

재귀적으로 계속 쪼개면 점화식이

$T(n)=7T(n/2)+O(n^2)$
이고, 해는

$T(n)=O(n^{\log_2 7}) \approx O(n^{2.807})$

  

#### 간단한 의사코드
```text
Strassen(A, B):
  if n <= threshold:
      return standard_multiply(A, B)

  A11,A12,A21,A22 = split(A)
  B11,B12,B21,B22 = split(B)

  compute M1..M7 (7 recursive multiplications)
  combine into C11..C22
  return join(C11..C22)

```

#### CLRS 스타일로 한단계 더 구성 예시

S1∼S10 (부분행렬의 합/차)

\begin{aligned} S_1 &= B_{12} - B_{22}\\ S_2 &= A_{11} + A_{12}\\ S_3 &= A_{21} + A_{22}\\ S_4 &= B_{21} - B_{11}\\ S_5 &= A_{11} + A_{22}\\ S_6 &= B_{11} + B_{22}\\ S_7 &= A_{12} - A_{22}\\ S_8 &= B_{21} + B_{22}\\ S_9 &= A_{11} - A_{21}\\ S_{10} &= B_{11} + B_{12} \end{aligned}


P1​∼P7​ (재귀적 곱 7회)

\begin{aligned} P_1 &= A_{11}\,S_1 \;=\; A_{11}(B_{12}-B_{22})\\ P_2 &= S_2\,B_{22} \;=\; (A_{11}+A_{12})B_{22}\\ P_3 &= S_3\,B_{11} \;=\; (A_{21}+A_{22})B_{11}\\ P_4 &= A_{22}\,S_4 \;=\; A_{22}(B_{21}-B_{11})\\ P_5 &= S_5\,S_6 \;=\; (A_{11}+A_{22})(B_{11}+B_{22})\\ P_6 &= S_7\,S_8 \;=\; (A_{12}-A_{22})(B_{21}+B_{22})\\ P_7 &= S_9\,S_{10} \;=\; (A_{11}-A_{21})(B_{11}+B_{12}) \end{aligned}

#### Flow
1. A11, A12등 각 변수에 맞는 matrix의 원소값 추출 및 할당(x) -> 오히려 리소스가 증가  
    A[i][j]가 𝑂(1)이라 딕셔너리로 바꾸는 게 오히려 오버헤드  
2. 그 다음에 파이썬 기본인 0 base로 할지, 1-base로 할지 결정  
    1-base로 하면 for loop를 통해 숫자를 하나씩 더해줘야 함.


In [29]:
A = [
    [1, 2],
    [3, 4]
]
B = [
    [5, 6],
    [7, 8]
]

In [34]:
# A = [[a11, a12],
#      [a21, a22]]
# B = [[b11, b12],
#      [b21, b22]]

n=2
C = [[0]* n for _ in range(n)]

M1 = (A[0][0] + A[1][1]) * (B[0][0] + B[1][1])
M2 = (A[1][0] + A[1][1]) *  B[0][0]
M3 =  A[0][0]            * (B[0][1] - B[1][1])
M4 =  A[1][1]            * (B[1][0] - B[0][0])
M5 = (A[0][0] + A[0][1]) *  B[1][1]
M6 = (A[1][0] - A[0][0]) * (B[0][0] + B[0][1])
M7 = (A[0][1] - A[1][1]) * (B[1][0] + B[1][1])

C[0][0] = M1 + M4 - M5 + M7
C[0][1] = M3 + M5
C[1][0] = M2 + M4
C[1][1] = M1 - M2 + M3 + M6

In [35]:
C

[[19, 22], [43, 50]]

---
아래는 참고

In [ ]:
# 일반적인 딕셔너리 형태로 값 매핑하기

A = [[1, 2],
     [3, 4]]

vars_dict = {}
for i, row in enumerate(A, start=1):      # 1-based
    for j, val in enumerate(row, start=1):
        key = f"a{i}{j}"                  # "a11", "a12", ...
        vars_dict[key] = val

print(vars_dict)        # {'a11': 1, 'a12': 2, 'a21': 3, 'a22': 4}
print(vars_dict["a12"]) # 2


{'a11': 1, 'a12': 2, 'a21': 3, 'a22': 4}
2


In [26]:
A = [[1, 2],
     [3, 4]]

vars_dict = {}
for i, k in enumerate(A, start=1):      # 1-based
    print(k)

[1, 2]
[3, 4]


In [24]:
vars_dict["a12"]

2

In [36]:
A = [[1, 2],
     [3, 4]]

for i, row in enumerate(A):            # i: 행 인덱스
    for j, val in enumerate(row):      # j: 열 인덱스
        print(i, j, val)


0 0 1
0 1 2
1 0 3
1 1 4


In [37]:
A = [[1, 2],
     [3, 4]]

for i, row in enumerate(A, start=1):            # i: 행 인덱스
    for j, val in enumerate(row, start=1):      # j: 열 인덱스
        print(i, j, val)


1 1 1
1 2 2
2 1 3
2 2 4
